In [19]:
from bs4 import BeautifulSoup
import re
import requests

In [14]:
%ls -lt

total 24
-rw-r--r--   1 bala  staff  3480 Nov 18 12:36 File-Pattern-CVE.ipynb
-rw-r--r--   1 bala  staff  2379 Nov 18 11:54 scrape.py
-rw-r--r--   1 bala  staff    36 Nov 18 08:54 README.md
drwxr-xr-x  31 bala  staff   992 Nov 15 22:10 cve-data/


In [15]:
file_base = %pwd
file_base += "/cve-data/"
file_base

'/Users/bala/Desktop/227/code/CSE-227/cve-data/'

In [68]:
CVE_IDS = []
file = "download ("
for i in range(0,29):
    file_path = file_base + file + str(i) + ").tsv"
    with open(file_path) as f:
        contents = f.readlines()
        for line in contents[1:]:
            words = line.split('\t')
            CVE_IDS.append(words[1])

In [71]:
CVE_TO_BUGID = dict()

for cve in CVE_IDS:
    url = "http://www.cvedetails.com/cve/" + cve + "/"
    hdr = {'User-Agent': 'Mozilla/5.0'}
    html = requests.get(url)
    soup = BeautifulSoup(html.text, "html.parser")
    
    table = soup.find("table", {"id": "vulnrefstable"})
    a_tags = table.find_all('a')
    for a in a_tags:
        if "bugzilla" in a['href']:
            if cve in CVE_TO_BUGID:
                CVE_TO_BUGID[cve].append(a['href'].split("=")[1])
            else:
                CVE_TO_BUGID[cve] = [a['href'].split("=")[1]]

# some CVE entries don't have bugzilla links
print(len(CVE_TO_BUGID))

1233


In [ ]:
FILE_FREQUENCY = dict()

for key_cve, value_bugs in CVE_TO_BUGID.items():
    for bug in value_bugs:
        s_url = "https://www.openhub.net/p/firefox/commits?query="+ str(bug)
        s_hdr = {'User-Agent': 'Mozilla/5.0'}
        s_html = requests.get(s_url)
        s_soup = BeautifulSoup(s_html.text, "html.parser")

        tags = s_soup('a')
        for t in tags:
            l = t.get('href', None)
            if l != None:
                commit = re.findall("commits/[0-9]+", l)
                if not commit:
                    continue
                t_url = "https://www.openhub.net/p/firefox/commits/" + commit[0].split("/")[1]
                t_req = urllib.request.Request(t_url, headers={'User-Agent': 'Mozilla/5.0'})
                t_html = urllib.request.urlopen(t_req).read()
                t_soup = BeautifulSoup(t_html, "lxml")
                rows = t_soup.findAll('tr', {'class': 'odd nohover'})
                if rows != []:
                    for row in rows:
                        file = row.find_all('td')[0].contents[0].strip()
                        if file in FILE_FREQUENCY:
                            FILE_FREQUENCY[file] += 1
                        else:
                            FILE_FREQUENCY[file] = 1
FILE_FREQUENCY